# Step 1: Create a Log File




In [66]:
log_data = """192.168.1.1 - - [03/Dec/2024:10:12:34 +0000] "GET /home HTTP/1.1" 200 512
203.0.113.5 - - [03/Dec/2024:10:12:35 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:12:36 +0000] "GET /about HTTP/1.1" 200 256
192.168.1.1 - - [03/Dec/2024:10:12:37 +0000] "GET /contact HTTP/1.1" 200 312
198.51.100.23 - - [03/Dec/2024:10:12:38 +0000] "POST /register HTTP/1.1" 200 128
203.0.113.5 - - [03/Dec/2024:10:12:39 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
192.168.1.100 - - [03/Dec/2024:10:12:40 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:12:41 +0000] "GET /dashboard HTTP/1.1" 200 1024
198.51.100.23 - - [03/Dec/2024:10:12:42 +0000] "GET /about HTTP/1.1" 200 256
192.168.1.1 - - [03/Dec/2024:10:12:43 +0000] "GET /dashboard HTTP/1.1" 200 1024
203.0.113.5 - - [03/Dec/2024:10:12:44 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
203.0.113.5 - - [03/Dec/2024:10:12:45 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
192.168.1.100 - - [03/Dec/2024:10:12:46 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:12:47 +0000] "GET /profile HTTP/1.1" 200 768
192.168.1.1 - - [03/Dec/2024:10:12:48 +0000] "GET /home HTTP/1.1" 200 512
198.51.100.23 - - [03/Dec/2024:10:12:49 +0000] "POST /feedback HTTP/1.1" 200 128
203.0.113.5 - - [03/Dec/2024:10:12:50 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
192.168.1.1 - - [03/Dec/2024:10:12:51 +0000] "GET /home HTTP/1.1" 200 512
198.51.100.23 - - [03/Dec/2024:10:12:52 +0000] "GET /about HTTP/1.1" 200 256
203.0.113.5 - - [03/Dec/2024:10:12:53 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
192.168.1.100 - - [03/Dec/2024:10:12:54 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:12:55 +0000] "GET /contact HTTP/1.1" 200 512
198.51.100.23 - - [03/Dec/2024:10:12:56 +0000] "GET /home HTTP/1.1" 200 512
192.168.1.100 - - [03/Dec/2024:10:12:57 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
203.0.113.5 - - [03/Dec/2024:10:12:58 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:12:59 +0000] "GET /dashboard HTTP/1.1" 200 1024
192.168.1.1 - - [03/Dec/2024:10:13:00 +0000] "GET /about HTTP/1.1" 200 256
198.51.100.23 - - [03/Dec/2024:10:13:01 +0000] "POST /register HTTP/1.1" 200 128
203.0.113.5 - - [03/Dec/2024:10:13:02 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
192.168.1.100 - - [03/Dec/2024:10:13:03 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:13:04 +0000] "GET /profile HTTP/1.1" 200 768
198.51.100.23 - - [03/Dec/2024:10:13:05 +0000] "GET /about HTTP/1.1" 200 256
192.168.1.1 - - [03/Dec/2024:10:13:06 +0000] "GET /home HTTP/1.1" 200 512
198.51.100.23 - - [03/Dec/2024:10:13:07 +0000] "POST /feedback HTTP/1.1" 200 128
"""

with open("sample.log", "w") as file:
    file.write(log_data)

print("Log file 'sample.log' has been created.")


Log file 'sample.log' has been created.


# Step 2: Import Necessary Modules

In [67]:
import re
import pandas as pd 
from collections import Counter
import csv


In [68]:
# Compile regex for parsing log lines
# The regex matches fields in a typical log entry: IP, HTTP method, endpoint, and status code.
LOG_PATTERN = re.compile(r'^(?P<ip>\S+) .* "(?P<method>\S+) (?P<endpoint>\S+) .*" (?P<status>\d+)')

# Generator to process the log file line by line
def process_log_file(log_file):
    """
    Reads a log file line by line using a generator to save memory.
    This approach is scalable for large log files.
    """
    try:
        with open(log_file, 'r') as file:
            for line in file:
                yield line.strip()  # Yields one line at a time
    except FileNotFoundError:
        print(f"Error: File '{log_file}' not found.")
        return
    except Exception as e:
        print(f"An error occurred: {e}")
        return

# Function to parse a log line using the compiled regex
def parse_log_line(line):
    """
    Parses a log line using the compiled regex pattern.
    Returns a dictionary with extracted fields or None if the line doesn't match.
    """
    match = LOG_PATTERN.match(line)
    if match:
        return match.groupdict()  # Returns a dictionary of extracted fields
    return None

# Step 3: Count Requests per IP Address:
### Parse the provided log file to extract all IP addresses.
### Calculate the number of requests made by each IP address.
### Sort and display the results in descending order of request counts.
### Example output:
#### IP Address           Request Count
#### 192.168.1.1          234
#### 203.0.113.5          187
#### 10.0.0.2             92

In [69]:
# Function to count requests per IP
def count_requests_per_ip(log_file):
    """
    Counts the number of requests made by each IP address in the log file.
    """
    ip_counter = Counter()
    for line in process_log_file(log_file):  # Processes one line at a time
        log_data = parse_log_line(line)
        if log_data:
            ip = log_data['ip']  # Extracts the IP address
            ip_counter[ip] += 1  # Increments the count for the IP

    # Output results in a formatted table
    print(f"{'IP Address':<20}{'Request Count':<15}")
    print("-" * 35)
    for ip, count in ip_counter.items():
        print(f"{ip:<20}{count:<15}")
    return ip_counter

# Identify the Most Frequently Accessed Endpoint:
### Extract the endpoints (e.g., URLs or resource paths) from the log file.
### Identify the endpoint accessed the highest number of times.
### Provide the endpoint name and its access count.
#### Example output:
#### Most Frequently Accessed Endpoint:
#### /home (Accessed 403 times)

In [70]:
# Function to identify the most frequently accessed endpoint
def most_frequent_endpoint(log_file):
    """
    Identifies the endpoint that is accessed most frequently.
    """
    endpoint_counter = Counter()
    for line in process_log_file(log_file):
        log_data = parse_log_line(line)
        if log_data:
            endpoint = log_data['endpoint']  # Extracts the endpoint
            endpoint_counter[endpoint] += 1  # Increments the count for the endpoint

    # Find and print the most accessed endpoint
    most_frequent = endpoint_counter.most_common(1)[0] if endpoint_counter else None
    if most_frequent:
        print("Most Frequently Accessed Endpoint:")
        print(f"{most_frequent[0]} (Accessed {most_frequent[1]} times)")
    return most_frequent


# Detect Suspicious Activity:
### Identify potential brute force login attempts by:
### Searching for log entries with failed login attempts (e.g., HTTP status code 401 or a specific failure message like "Invalid credentials").
### Flagging IP addresses with failed login attempts exceeding a configurable threshold (default: 10 attempts).
### Display the flagged IP addresses and their failed login counts.
#### Example output:
#### Suspicious Activity Detected:
#### IP Address           Failed Login Attempts
#### 192.168.1.100        56
#### 203.0.113.34         12

In [71]:
# Function to detect suspicious activity
def detect_suspicious_activity(log_file, threshold=10):
    """
    Detects suspicious IP addresses based on failed login attempts.
    An IP is flagged if it has more failed attempts than the threshold.
    """
    failed_attempts = Counter()
    for line in process_log_file(log_file):
        log_data = parse_log_line(line)
        if log_data and log_data['status'] == '401':  # Failed login status
            ip = log_data['ip']
            failed_attempts[ip] += 1  # Count failed attempts for the IP

    # Filter IPs exceeding the threshold
    flagged_ips = {ip: count for ip, count in failed_attempts.items() if count > threshold}
    if flagged_ips:
        print("Suspicious Activity Detected:")
        print(f"{'IP Address':<20}{'Failed Login Attempts':<25}")
        print("-" * 45)
        for ip, count in flagged_ips.items():
            print(f"{ip:<20}{count:<25}")
    else:
        print("No suspicious activity detected.")
    return flagged_ips


# Step 3: **Output Results**:
### Display the results in a clear, organized format in the terminal.
### Save the results to a CSV file named `log_analysis_results.csv` with the following structure:
#### Requests per IP**: Columns: `IP Address`, `Request Count`
#### Most Accessed Endpoint: Columns: `Endpoint`, `Access Count`
#### Suspicious Activity: Columns: `IP Address`, `Failed Login Count`

In [72]:
# Function to save results to a CSV file
def save_to_csv(results, output_file):
    """
    Saves the analysis results to a CSV file with separate sections for:
    - Requests per IP
    - Most accessed endpoint
    - Suspicious activity
    """
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)

        # Write Requests per IP
        writer.writerow(["Requests per IP"])
        writer.writerow(["IP Address", "Request Count"])
        for ip, count in results['ip_requests'].items():
            writer.writerow([ip, count])

        # Write Most Accessed Endpoint
        writer.writerow([])  # Blank line
        writer.writerow(["Most Accessed Endpoint"])
        if results['most_accessed_endpoint']:
            writer.writerow(["Endpoint", "Access Count"])
            writer.writerow(results['most_accessed_endpoint'])

        # Write Suspicious Activity
        writer.writerow([])  # Blank line
        writer.writerow(["Suspicious Activity"])
        writer.writerow(["IP Address", "Failed Login Count"])
        for ip, count in results['suspicious_activity'].items():
            writer.writerow([ip, count])


In [73]:
# Main function to run the analysis
def main():
    """
    Main function to perform log analysis and save the results.
    """
    log_file = "sample.log"  # Replace with your log file name
    threshold = 5  # Define threshold for suspicious activity

    # Run analyses
    ip_requests = count_requests_per_ip(log_file)
    most_accessed_endpoint = most_frequent_endpoint(log_file)
    suspicious_activity = detect_suspicious_activity(log_file, threshold)

    # Save results to a CSV file
    results = {
        'ip_requests': ip_requests,
        'most_accessed_endpoint': most_accessed_endpoint,
        'suspicious_activity': suspicious_activity
    }
    save_to_csv(results, "log_analysis_results.csv")
    print("\nResults saved to 'log_analysis_results.csv'.")

# Execute the main function when the script is run
if __name__ == "__main__":
    main()


IP Address          Request Count  
-----------------------------------
192.168.1.1         7              
203.0.113.5         8              
10.0.0.2            6              
198.51.100.23       8              
192.168.1.100       5              
Most Frequently Accessed Endpoint:
/login (Accessed 13 times)
Suspicious Activity Detected:
IP Address          Failed Login Attempts    
---------------------------------------------
203.0.113.5         8                        

Results saved to 'log_analysis_results.csv'.
